## САМОСТОЯТЕЛЬНАЯ РАБОТА

## Оценка продуктовой фичи

#### Задачи: 
- Посчитать показатели Retention Rate седьмого дня и ARPPU за два периода: до и после внедрения фичи.
- Сформировать выводы по анализу.

#### Примечание: 
- т.к.в задании нет конкретного пояснения, каких пользователей учитывать, буду считать так, как представлено в модуле OD-2,т.е. "коэффициент удержания" и среднюю выручку от платящего пользователя в мае - для пользователей, которые зарегистрировались в мае, а в июне только тех, кто зарегистрировался в июне.

#### Критерии оценки:

- Отсутствие ошибок: ноутбук открывается без ошибок.
- Retention Rate седьмого дня: показатель рассчитан для периодов до и после внедрения фичи.
- ARPPU: показатель рассчитан для периодов до и после внедрения фичи.
- Представлены выводы, сделанные на основе анализа.

#### Требованиях к ответу:

- Ответ должен быть в виде ipynb-файла.
- Ipynb-файл должен запускаться и выполняться без необходимости изменять порядок действий. То есть при нажатии кнопки Restart Kernel & Re-run the whole notebook ноутбук должен выполниться без ошибок и выдать в конце результат проверки второй гипотезы.
- Результатом выполнения задания является написанный алгоритм, который производит вычисления и выводит данные, на основе которых можно подтвердить или опровергнуть гипотезу.
- Обязательно наличие выводов в формате ячеек с Markdown.
- Большим плюсом (но это не обязательное условие) будет, если в ходе алгоритма даны комментарии к коду (в виде комментариев Python или в разметке Markdown в Jupyter Notebook) с описанием назначения создаваемых переменных и выражений.

In [1]:
! pip install psycopg2

In [2]:
# Импортируем необходимые библиотеки
%matplotlib inline
import pandas as pd
import numpy as np
import psycopg2  
import psycopg2.extras 

#### Задача 1
Посчитать показатели Retention Rate седьмого дня за два периода: до и после внедрения фичи.

In [3]:
# Создадим функцию getRegisteredUsersCount(), в которой выполним SQL-запрос.
# Подсчитаем количество зарегистрировавшихся пользователей за май-июнь 2019 года в разрезе дней регистрации.
# Запишем полученные данные в датафрейм.

def getRegisteredUsersCount():
    query = '''SELECT r.registration_date, count(distinct(r.user_id)) as registered_users_count
    FROM case9.registration r
    WHERE r.registration_date >= '2019-05-01' and r.registration_date < '2019-07-01'
    GROUP BY 1
    '''.format()
    conn = psycopg2.connect("dbname='skillfactory' user='skillfactory' host='84.201.134.129' password='cCkxxLVrDE8EbvjueeMedPKt' port=5432")
    dict_cur = conn.cursor(cursor_factory=psycopg2.extras.DictCursor)
    dict_cur.execute(query)
    rows = dict_cur.fetchall()
    data = []
    for row in rows:
        data.append(dict(row))
    return data
registered_users_count = pd.DataFrame(getRegisteredUsersCount())

In [4]:
# Посмотрим на данные
registered_users_count.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 61 entries, 0 to 60
Data columns (total 2 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   registration_date       61 non-null     object
 1   registered_users_count  61 non-null     int64 
dtypes: int64(1), object(1)
memory usage: 1.1+ KB


In [5]:
registered_users_count.head()

,registration_date,registered_users_count
0,2019-05-01,6516
1,2019-05-02,5309
2,2019-05-03,6266
3,2019-05-04,8639
4,2019-05-05,9787


In [6]:
# Преобразуем столбец с датой в тип datetime
registered_users_count['registration_date']=pd.to_datetime(registered_users_count['registration_date'])

In [7]:
# Получим данные об активных пользователях 
# и подсчитаем по дням количество активных пользователей с указанием даты регистрации.
# дата регистрации будет выступать в качестве когорты. 
# Создадим функцию getActiveUsersCountWithCohorts() 
# Запишем данные, полученные с помощью этой функции, в датафрейм active_users_count_with_cohorts.

def getActiveUsersCountWithCohorts():
    query = '''SELECT ac.activity_date, r.registration_date, count(distinct(ac.user_id)) as active_users_count
    FROM case9.user_activity ac
    LEFT JOIN case9.registration r ON r.user_id = ac.user_id
    WHERE r.registration_date >= '2019-05-01' and r.registration_date < '2019-07-01'
    GROUP BY 1,2
    '''.format()
    conn = psycopg2.connect("dbname='skillfactory' user='skillfactory' host='84.201.134.129' password='cCkxxLVrDE8EbvjueeMedPKt' port=5432")
    dict_cur = conn.cursor(cursor_factory=psycopg2.extras.DictCursor)
    dict_cur.execute(query)
    rows = dict_cur.fetchall()
    data = []
    for row in rows:
        data.append(dict(row))
    return data 
active_users_count_with_cohorts = pd.DataFrame(getActiveUsersCountWithCohorts())

In [8]:
# Посмотрим на данные
active_users_count_with_cohorts.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1891 entries, 0 to 1890
Data columns (total 3 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   activity_date       1891 non-null   object
 1   registration_date   1891 non-null   object
 2   active_users_count  1891 non-null   int64 
dtypes: int64(1), object(2)
memory usage: 44.4+ KB


In [9]:
active_users_count_with_cohorts.head()

,activity_date,registration_date,active_users_count
0,2019-05-01,2019-05-01,2590
1,2019-05-02,2019-05-01,2146
2,2019-05-02,2019-05-02,2049
3,2019-05-03,2019-05-01,1637
4,2019-05-03,2019-05-02,1817


In [10]:
# Преобразуем столбцы с датой в тип datetime
for col in ['registration_date', 'activity_date']:
    active_users_count_with_cohorts[col]=pd.to_datetime(active_users_count_with_cohorts[col])

In [11]:
# Объединим таблицы, чтобы рассчитать процент вернувшихся пользователей на определённый день
retention_table=active_users_count_with_cohorts.merge(registered_users_count, on='registration_date', how='left')
retention_table.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1891 entries, 0 to 1890
Data columns (total 4 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   activity_date           1891 non-null   datetime64[ns]
 1   registration_date       1891 non-null   datetime64[ns]
 2   active_users_count      1891 non-null   int64         
 3   registered_users_count  1891 non-null   int64         
dtypes: datetime64[ns](2), int64(2)
memory usage: 73.9 KB


In [12]:
# Посмотрим на первые строки данных
retention_table.head()

,activity_date,registration_date,active_users_count,registered_users_count
0,2019-05-01,2019-05-01,2590,6516
1,2019-05-02,2019-05-01,2146,6516
2,2019-05-02,2019-05-02,2049,5309
3,2019-05-03,2019-05-01,1637,6516
4,2019-05-03,2019-05-02,1817,5309


In [13]:
# Рассчитаем какое количество дней прошло с момента регистрации до определённого дня активности
retention_table['lifetime']=retention_table['activity_date']-retention_table['registration_date']

# Чтобы получить целое количество дней, значения в колонке с типом timedelta разделим на константу np.timedelta64(1,'D') 
retention_table['lifetime']=retention_table['lifetime']/np.timedelta64(1,'D')

# Приведем тип к целому числу
retention_table['lifetime']=retention_table['lifetime'].astype(int)
retention_table.head()

,activity_date,registration_date,active_users_count,registered_users_count,lifetime
0,2019-05-01,2019-05-01,2590,6516,0
1,2019-05-02,2019-05-01,2146,6516,1
2,2019-05-02,2019-05-02,2049,5309,0
3,2019-05-03,2019-05-01,1637,6516,2
4,2019-05-03,2019-05-02,1817,5309,1


In [14]:
# Чтобы посчитать ARPPU, напишем SQL-запрос, который позволит получить выручку по дням в разрезе когорт для пользователей,
# зарегистрировавшихся в мае-июне 2019 года. Создадим функцию getRevenue()
# запишем данные в датафрейм revenue.

def getRevenue():
    query = '''SELECT re.date, r.registration_date, count(distinct(re.user_id)) as users_count_with_revenue, sum(re.revenue) as revenue
    FROM case9.user_revenue re
    LEFT JOIN case9.registration r ON r.user_id = re.user_id
    WHERE r.registration_date >= '2019-05-01' and r.registration_date < '2019-07-01'
    GROUP BY 1,2
    '''.format()
    conn = psycopg2.connect("dbname='skillfactory' user='skillfactory' host='84.201.134.129' password='cCkxxLVrDE8EbvjueeMedPKt' port=5432")
    dict_cur = conn.cursor(cursor_factory=psycopg2.extras.DictCursor)
    dict_cur.execute(query)
    rows = dict_cur.fetchall()
    data = []
    for row in rows:
        data.append(dict(row))
    return data
revenue = pd.DataFrame(getRevenue())

In [15]:
# Посмотрим на данные
revenue.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1745 entries, 0 to 1744
Data columns (total 4 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   date                      1745 non-null   object 
 1   registration_date         1745 non-null   object 
 2   users_count_with_revenue  1745 non-null   int64  
 3   revenue                   1745 non-null   float64
dtypes: float64(1), int64(1), object(2)
memory usage: 54.7+ KB


In [16]:
#Преобразуем дату в формат даты
for col in ['date', 'registration_date']:
    revenue[col]=pd.to_datetime(revenue[col])
revenue.head()

,date,registration_date,users_count_with_revenue,revenue
0,2019-05-01,2019-05-01,1672,2357.580
1,2019-05-02,2019-05-01,1420,514.679
2,2019-05-02,2019-05-02,1367,1629.140
3,2019-05-03,2019-05-01,1048,390.952
4,2019-05-03,2019-05-02,1164,533.608


In [17]:
# Перед объединением таблиц переименуем столбей date в activity_date
revenue=revenue.rename(columns={'date':'activity_date'})
revenue.head()

,activity_date,registration_date,users_count_with_revenue,revenue
0,2019-05-01,2019-05-01,1672,2357.580
1,2019-05-02,2019-05-01,1420,514.679
2,2019-05-02,2019-05-02,1367,1629.140
3,2019-05-03,2019-05-01,1048,390.952
4,2019-05-03,2019-05-02,1164,533.608


In [18]:
# Объединим таблицу retention_table и revenue
retention_table_with_revenue=retention_table.merge(revenue, on=['activity_date','registration_date'], how='left')
retention_table_with_revenue.head()

,activity_date,registration_date,active_users_count,registered_users_count,lifetime,users_count_with_revenue,revenue
0,2019-05-01,2019-05-01,2590,6516,0,1672.0,2357.580
1,2019-05-02,2019-05-01,2146,6516,1,1420.0,514.679
2,2019-05-02,2019-05-02,2049,5309,0,1367.0,1629.140
3,2019-05-03,2019-05-01,1637,6516,2,1048.0,390.952
4,2019-05-03,2019-05-02,1817,5309,1,1164.0,533.608


In [19]:
retention_table_with_revenue.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1891 entries, 0 to 1890
Data columns (total 7 columns):
 #   Column                    Non-Null Count  Dtype         
---  ------                    --------------  -----         
 0   activity_date             1891 non-null   datetime64[ns]
 1   registration_date         1891 non-null   datetime64[ns]
 2   active_users_count        1891 non-null   int64         
 3   registered_users_count    1891 non-null   int64         
 4   lifetime                  1891 non-null   int32         
 5   users_count_with_revenue  1745 non-null   float64       
 6   revenue                   1745 non-null   float64       
dtypes: datetime64[ns](2), float64(2), int32(1), int64(2)
memory usage: 110.8 KB


In [20]:
# В столбцах user_count_with_revenue и revenue есть пропущенные значения -это говорит о том, что в какие-то даты у каких-то
# когорт не было оплат.
# Заменим пропущенные значения на 0
for col in ['users_count_with_revenue', 'revenue']:
    retention_table_with_revenue[col]=retention_table_with_revenue[col].fillna(0)

In [21]:
retention_table_with_revenue.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1891 entries, 0 to 1890
Data columns (total 7 columns):
 #   Column                    Non-Null Count  Dtype         
---  ------                    --------------  -----         
 0   activity_date             1891 non-null   datetime64[ns]
 1   registration_date         1891 non-null   datetime64[ns]
 2   active_users_count        1891 non-null   int64         
 3   registered_users_count    1891 non-null   int64         
 4   lifetime                  1891 non-null   int32         
 5   users_count_with_revenue  1891 non-null   float64       
 6   revenue                   1891 non-null   float64       
dtypes: datetime64[ns](2), float64(2), int32(1), int64(2)
memory usage: 110.8 KB


In [22]:
# Выделим данные активности пользавателей на 7 день после регистрации (май 2019)
may_retention_rate_7_table=retention_table_with_revenue[(retention_table_with_revenue['lifetime']==7)&\
(retention_table_with_revenue['registration_date']>='2019-05-01')&\
(retention_table_with_revenue['registration_date']<'2019-06-01')]

# Посчитаем Retention Rate 7 дня активности для пользователей, зарегистрировавшихся в мае 2019 
may_retention_rate_7=round(may_retention_rate_7_table['active_users_count'].sum()/\
may_retention_rate_7_table['registered_users_count'].sum()*100, 2)
print('Retention Rate 7 дня активности пользователей, зарегистрировавшихся в мае 2019, составляет {} %'.format(may_retention_rate_7))


Retention Rate 7 дня активности пользователей, зарегистрировавшихся в мае 2019, составляет 12.93 %


In [23]:
# Выделим данные активности пользавателей на 7 день после регистрации (июнь 2019)
june_retention_rate_7_table=retention_table_with_revenue[(retention_table_with_revenue['lifetime']==7)&\
(retention_table_with_revenue['registration_date']>='2019-06-01')&(retention_table_with_revenue['registration_date']<'2019-07-01')]

#june_retention_rate_7_table
# Посчитаем Retention Rate 7 дня активности для пользователей, зарегистрировавшихся в июне 2019
june_retention_rate_7=round(june_retention_rate_7_table['active_users_count'].sum()/june_retention_rate_7_table['registered_users_count'].sum()*100, 2)
print('Retention Rate 7 дня активности пользователей, зарегистрировавшихся в июне 2019, составляет {} %'.format(june_retention_rate_7))

Retention Rate 7 дня активности пользователей, зарегистрировавшихся в июне 2019, составляет 12.56 %


### Задача 2
Посчитать показатели ARPPU за два периода: до и после внедрения фичи.

In [24]:
# Отфильтруем данные с регистрацией пользователей в мае 2019
may_retention_table_with_revenue=retention_table_with_revenue[(retention_table_with_revenue['registration_date']>='2019-05-01')&\
(retention_table_with_revenue['registration_date']<'2019-06-01')]
may_retention_table_with_revenue.head()

,activity_date,registration_date,active_users_count,registered_users_count,lifetime,users_count_with_revenue,revenue
0,2019-05-01,2019-05-01,2590,6516,0,1672.0,2357.580
1,2019-05-02,2019-05-01,2146,6516,1,1420.0,514.679
2,2019-05-02,2019-05-02,2049,5309,0,1367.0,1629.140
3,2019-05-03,2019-05-01,1637,6516,2,1048.0,390.952
4,2019-05-03,2019-05-02,1817,5309,1,1164.0,533.608


In [25]:
# Посчитаем показатель ARPPU для пользователей, зарегистрировавшихся в мае 2019
may_ARPPU=round(may_retention_table_with_revenue['revenue'].sum()/may_retention_table_with_revenue['users_count_with_revenue'].sum(),2)
print('ARPPU для пользователей, зарегистрировавшихся в мае 2019 - {} y.e.'.format(may_ARPPU))

ARPPU для пользователей, зарегистрировавшихся в мае 2019 - 0.31 y.e.


In [26]:
# Отфильтруем данные с регистрацией пользователей в июне 2019
june_retention_table_with_revenue=retention_table_with_revenue[(retention_table_with_revenue['registration_date']>='2019-06-01')&\
(retention_table_with_revenue['registration_date']<'2019-07-01')]
june_retention_table_with_revenue.head()

,activity_date,registration_date,active_users_count,registered_users_count,lifetime,users_count_with_revenue,revenue
526,2019-06-01,2019-06-01,1651,4833,0,1082.0,1611.860
556,2019-06-02,2019-06-01,1429,4833,1,928.0,317.738
557,2019-06-02,2019-06-02,1933,5255,0,1257.0,1305.220
586,2019-06-03,2019-06-01,1021,4833,2,623.0,192.273
587,2019-06-03,2019-06-02,1550,5255,1,1008.0,554.858


In [27]:
# Посчитаем показатель ARPPU для пользователей, зарегистрировавшихся в июне 2019
june_ARPPU=round(june_retention_table_with_revenue['revenue'].sum()/june_retention_table_with_revenue['users_count_with_revenue'].sum(),2)
print('ARPPU для пользователей, зарегистрировавшихся в июне 2019 - {} y.e.'.format(june_ARPPU))

ARPPU для пользователей, зарегистрировавшихся в июне 2019 - 0.39 y.e.


In [28]:
# Посчитаем во сколько раз увеличился ARPPU в июне по сравнению с маем
round(june_ARPPU/may_ARPPU,2)


1.26

#### Выводы:
1. Показатель Retention Rate седьмого дня после внедрения фичи незначительно ниже, чем до ее внедрения:
    
        -Retention Rate 7 дня активности пользователей, зарегистрировавшихся в мае 2019, составляет 12.93 % 
        -Retention Rate 7 дня активности пользователей, зарегистрировавшихся в июне 2019, составляет 12.56 %
            
2. Показатель ARPPU после внедрения фичи (июнь 2019) вырос в 1.26 раза (на 26%) по сравнению с таким же периодом до введения фичи (май 2019):
        
        -ARPPU для пользователей, зарегистрировавшихся в мае 2019 - 0.31 y.e.
        -ARPPU для пользователей, зарегистрировавшихся в июне 2019 - 0.39 y.e.
       
3. По имеющимся данным можно сделать вывод, что в первый месяц внедрения фичи коэффициент удержания клиентов практически не изменился, однако на 26% увеличился средний доход с активного (платящего) пользователя. В первый месяц фича оказалось эффективной, нужно оценивать в динамике.